-------------------

In [1]:
import random
import time
import pandas as pd
from bs4 import BeautifulSoup as bs
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import undetected_chromedriver as uc

# Configura el navegador con undetected_chromedriver
browser = uc.Chrome()

# URL inicial: Página de listados de Madrid
base_url = "https://www.pisos.com/venta/pisos-madrid/"
browser.get(base_url)

# Manejar las cookies
try:
    WebDriverWait(browser, 20).until(
        EC.element_to_be_clickable((By.XPATH, '//*[@id="didomi-notice-agree-button"]'))
    ).click()
except Exception:
    print("No se encontró el botón de cookies o ya fue aceptado.")

# Lista para almacenar los datos
all_data = []
inmueble_counter = 1  # Contador de inmuebles

# Función para extraer datos de un inmueble dado su enlace
def extraer_datos_inmueble(enlace):
    try:
        browser.get(enlace)
        WebDriverWait(browser, 20).until(EC.presence_of_element_located((By.TAG_NAME, 'body')))
        html_inmueble = browser.page_source
        soup_inmueble = bs(html_inmueble, 'lxml')

        # Extraer datos del inmueble
        try:
            # Precio
            precio_element = soup_inmueble.find('div', {'class': 'price__value jsPriceValue'})
            precio = precio_element.text.split(' ')[0] if precio_element else "N/A"

            # Superficie construida
            superficie_element = soup_inmueble.find('span', {'class': 'features__value'})
            superficie_construida = superficie_element.text.split(' ')[0] if superficie_element else "N/A"

            # Última actualización
            ultima_actualizacion_element = soup_inmueble.find('p', {'class': 'last-update__date'})
            ultima_actualizacion = ultima_actualizacion_element.text if ultima_actualizacion_element else "N/A"

            # Características
            c1 = soup_inmueble.find('div', {'class': 'features__content'})
            features_list = []
            if c1:
                features = c1.find_all('div', {'class': 'features__feature'})
                for feature in features:
                    label = feature.find('span', {'class': 'features__label'})
                    value = feature.find('span', {'class': 'features__value'})
                    if label and value:
                        features_list.append((label.get_text(strip=True), value.get_text(strip=True)))
            else:
                features_list = "N/A"

            # Certificado energético
            energy_certificate = soup_inmueble.find('div', {'class': 'details__block energy-certificate'})
            if energy_certificate:
                consumo_element = energy_certificate.find('div', {'class': 'energy-certificate__data'})
                consumo = consumo_element.find_all('span')[1].get_text(strip=True) if consumo_element else "N/A"
                
                emisiones_element = energy_certificate.find_all('div', {'class': 'energy-certificate__data'})
                emisiones = emisiones_element[1].find_all('span')[1].get_text(strip=True) if len(emisiones_element) > 1 else "N/A"
            else:
                consumo = "N/A"
                emisiones = "N/A"

            # Guarda los datos en una lista
            data = {
                'Enlace': enlace,
                'Precio': precio,
                'Superficie Construida': superficie_construida,
                'Última Actualización': ultima_actualizacion,
                'Consumo Energético': consumo,
                'Emisiones CO2': emisiones,
                'Características': features_list,
                'Tipo de operación': 'Alquiler'
            }

            return data

        except Exception as e:
            print(f"Error al extraer datos de {enlace}: {e}")
            return None

    except Exception as e:
        print(f"Error al cargar la página del inmueble: {e}")
        return None

# Bucle para iterar sobre múltiples páginas de resultados hasta alcanzar los 300 inmuebles
page_number = 1
max_pages = 500  # Aumentado a 500 para asegurar 300 inmuebles, incluso si hay solo uno por página
while len(all_data) < 300 and page_number <= max_pages:
    try:
        # Cargar la página actual
        page_url = f"{base_url}pagina-{page_number}/"
        browser.get(page_url)
        WebDriverWait(browser, 20).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, '.ad-preview'))
        )

        # Obtener todos los enlaces de la página actual
        html = browser.page_source
        soup = bs(html, 'lxml')
        inmuebles = soup.find_all('div', {'class': 'ad-preview'})

        # Extraer datos de cada inmueble de la página
        for inmueble in inmuebles:
            enlace = inmueble.get('data-lnk-href')
            if enlace:
                enlace_completo = f"https://www.pisos.com{enlace}"
                data = extraer_datos_inmueble(enlace_completo)
                if data:
                    all_data.append(data)
                    # Imprimir los datos del inmueble con el contador
                    print(f"Inmueble {inmueble_counter}: {data}")
                    inmueble_counter += 1

                    # Verificar si se ha alcanzado el límite de 300 inmuebles
                    if len(all_data) >= 300:
                        break

                # Pausa para evitar ser bloqueado
                time.sleep(random.uniform(5, 8))

        # Imprimir el progreso
        print(f"Procesada la página {page_number}. Total de inmuebles recogidos: {len(all_data)}")

        # Pasar a la siguiente página si aún no se ha alcanzado el límite
        page_number += 1

    except Exception as e:
        print(f"Error al procesar la página {page_number}: {e}")
        break

# Convertir los datos en un DataFrame de pandas y guardarlos
df1 = pd.DataFrame(all_data)
print(df1)

# Opcional: guarda el DataFrame en un archivo CSV
df1.to_csv('muestra_compra_300_inmuebles.csv', index=False)

# Cierra el navegador
browser.quit()


Inmueble 1: {'Enlace': 'https://www.pisos.com/comprar/atico-nino_jesus28007-45066276232_997608/', 'Precio': '900.000', 'Superficie Construida': '160', 'Última Actualización': 'Anuncio actualizado el 25/10/2024', 'Consumo Energético': 'N/A', 'Emisiones CO2': 'N/A', 'Características': [('Superficie construida:', '160 m²'), ('Superficie útil:', '102 m²'), ('Habitaciones:', '1'), ('Baños:', '2'), ('Planta:', '2ª'), ('Antigüedad:', 'Más de 50 años'), ('Conservación:', 'Reformado'), ('Gastos de comunidad:', 'Entre 20 y 40 €'), ('Referencia:', '4997608-0120-00318')], 'Tipo de operación': 'Alquiler'}
Inmueble 2: {'Enlace': 'https://www.pisos.com/comprar/piso-zofio28026-45065797739_440000/', 'Precio': '198.000', 'Superficie Construida': '71', 'Última Actualización': 'Anuncio actualizado el 22/10/2024', 'Consumo Energético': 'Consumo:300 kWh/m² año', 'Emisiones CO2': 'Emisiones:65 Kg CO₂/m² año', 'Características': [('Superficie construida:', '71 m²'), ('Superficie útil:', '66 m²'), ('Habitacion

KeyboardInterrupt: 

In [2]:
df1

,Enlace,Precio,Superficie Construida,Última Actualización,Consumo Energético,Emisiones CO2,Características,Tipo de operación
0,https://www.pisos.com/comprar/chalet_pareado-s...,374.000,264,Anuncio actualizado el 25/10/2024,N/A,N/A,"[(Superficie construida:, 264 m²), (Superficie...",Alquiler
1,https://www.pisos.com/comprar/piso-goya28009-4...,536.000,73,Anuncio actualizado el 25/10/2024,Consumo:260 kWh/m² año,Emisiones:53 Kg CO₂/m² año,"[(Superficie construida:, 73 m²), (Habitacione...",Alquiler
2,https://www.pisos.com/comprar/atico-nino_jesus...,900.000,160,Anuncio actualizado el 25/10/2024,N/A,N/A,"[(Superficie construida:, 160 m²), (Superficie...",Alquiler
3,https://www.pisos.com/comprar/casa_pareada-cub...,320.000,162,Anuncio actualizado el 22/10/2024,N/A,N/A,"[(Superficie construida:, 162 m²), (Superficie...",Alquiler
4,https://www.pisos.com/comprar/chalet-el_olivar...,2.370.000,750,Anuncio actualizado el 25/10/2024,N/A,N/A,"[(Superficie construida:, 750 m²), (Superficie...",Alquiler
...,...,...,...,...,...,...,...,...
295,https://www.pisos.com/comprar/chalet-mirasierr...,2.900.000,563,Anuncio actualizado el 25/10/2024,N/A,N/A,"[(Superficie construida:, 563 m²), (Habitacion...",Alquiler
296,https://www.pisos.com/comprar/casa-torrelodone...,1.175.000,400,Anuncio actualizado el 25/10/2024,Consumo:80 kWh/m² año,Emisiones:67 Kg CO₂/m² año,"[(Superficie construida:, 400 m²), (Superficie...",Alquiler
297,https://www.pisos.com/comprar/chalet-zona_aven...,1.629.900,324,Anuncio actualizado el 21/10/2024,N/A,N/A,"[(Superficie construida:, 324 m²), (Habitacion...",Alquiler
298,https://www.pisos.com/comprar/chalet-castillo_...,1.150.000,580,Anuncio actualizado el 04/06/2024,N/A,N/A,"[(Superficie construida:, 580 m²), (Superficie...",Alquiler


In [ ]:
pip install undetected-chromedriver lxml

In [2]:
import random
import time
import pandas as pd
from bs4 import BeautifulSoup as bs
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import undetected_chromedriver as uc

# Configura el navegador con undetected_chromedriver
browser = uc.Chrome()

# URL inicial: Página de listados de Madrid
base_url = "https://www.pisos.com/alquiler/pisos-madrid/"
browser.get(base_url)

# Manejar las cookies
try:
    WebDriverWait(browser, 20).until(
        EC.element_to_be_clickable((By.XPATH, '//*[@id="didomi-notice-agree-button"]'))
    ).click()
except Exception:
    print("No se encontró el botón de cookies o ya fue aceptado.")

# Lista para almacenar los datos
all_data = []
inmueble_counter = 1  # Contador de inmuebles

# Función para extraer datos de un inmueble dado su enlace
def extraer_datos_inmueble(enlace):
    try:
        browser.get(enlace)
        WebDriverWait(browser, 20).until(EC.presence_of_element_located((By.TAG_NAME, 'body')))
        html_inmueble = browser.page_source
        soup_inmueble = bs(html_inmueble, 'lxml')

        # Extraer datos del inmueble
        try:
            # Precio
            precio_element = soup_inmueble.find('div', {'class': 'price__value jsPriceValue'})
            precio = precio_element.text.split(' ')[0] if precio_element else "N/A"

            # Superficie construida
            superficie_element = soup_inmueble.find('span', {'class': 'features__value'})
            superficie_construida = superficie_element.text.split(' ')[0] if superficie_element else "N/A"

            # Última actualización
            ultima_actualizacion_element = soup_inmueble.find('p', {'class': 'last-update__date'})
            ultima_actualizacion = ultima_actualizacion_element.text if ultima_actualizacion_element else "N/A"

            # Características
            c1 = soup_inmueble.find('div', {'class': 'features__content'})
            features_list = []
            if c1:
                features = c1.find_all('div', {'class': 'features__feature'})
                for feature in features:
                    label = feature.find('span', {'class': 'features__label'})
                    value = feature.find('span', {'class': 'features__value'})
                    if label and value:
                        features_list.append((label.get_text(strip=True), value.get_text(strip=True)))
            else:
                features_list = "N/A"

            # Certificado energético
            energy_certificate = soup_inmueble.find('div', {'class': 'details__block energy-certificate'})
            if energy_certificate:
                consumo_element = energy_certificate.find('div', {'class': 'energy-certificate__data'})
                consumo = consumo_element.find_all('span')[1].get_text(strip=True) if consumo_element else "N/A"
                
                emisiones_element = energy_certificate.find_all('div', {'class': 'energy-certificate__data'})
                emisiones = emisiones_element[1].find_all('span')[1].get_text(strip=True) if len(emisiones_element) > 1 else "N/A"
            else:
                consumo = "N/A"
                emisiones = "N/A"

            # Guarda los datos en una lista
            data = {
                'Enlace': enlace,
                'Precio': precio,
                'Superficie Construida': superficie_construida,
                'Última Actualización': ultima_actualizacion,
                'Consumo Energético': consumo,
                'Emisiones CO2': emisiones,
                'Características': features_list,
                'Tipo de operación': 'Alquiler'
            }

            return data

        except Exception as e:
            print(f"Error al extraer datos de {enlace}: {e}")
            return None

    except Exception as e:
        print(f"Error al cargar la página del inmueble: {e}")
        return None

# Bucle para iterar sobre múltiples páginas de resultados hasta alcanzar los 300 inmuebles
page_number = 1
max_pages = 500  # Aumentado a 500 para asegurar 300 inmuebles, incluso si hay solo uno por página
while len(all_data) < 300 and page_number <= max_pages:
    try:
        # Cargar la página actual
        page_url = f"{base_url}pagina-{page_number}/"
        browser.get(page_url)
        WebDriverWait(browser, 20).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, '.ad-preview'))
        )

        # Obtener todos los enlaces de la página actual
        html = browser.page_source
        soup = bs(html, 'lxml')
        inmuebles = soup.find_all('div', {'class': 'ad-preview'})

        # Extraer datos de cada inmueble de la página
        for inmueble in inmuebles:
            enlace = inmueble.get('data-lnk-href')
            if enlace:
                enlace_completo = f"https://www.pisos.com{enlace}"
                data = extraer_datos_inmueble(enlace_completo)
                if data:
                    all_data.append(data)
                    # Imprimir los datos del inmueble con el contador
                    print(f"Inmueble {inmueble_counter}: {data}")
                    inmueble_counter += 1

                    # Verificar si se ha alcanzado el límite de 300 inmuebles
                    if len(all_data) >= 300:
                        break

                # Pausa para evitar ser bloqueado
                time.sleep(random.uniform(5, 8))

        # Imprimir el progreso
        print(f"Procesada la página {page_number}. Total de inmuebles recogidos: {len(all_data)}")

        # Pasar a la siguiente página si aún no se ha alcanzado el límite
        page_number += 1

    except Exception as e:
        print(f"Error al procesar la página {page_number}: {e}")
        break

# Convertir los datos en un DataFrame de pandas y guardarlos
df = pd.DataFrame(all_data)
print(df)

# Opcional: guarda el DataFrame en un archivo CSV
df.to_csv('muestra_alquiler_300_inmuebles.csv', index=False)

# Cierra el navegador
browser.quit()


Inmueble 1: {'Enlace': 'https://www.pisos.com/alquilar/casa_unifamiliar-las_matas_los_penascales-29189878080_280500/', 'Precio': '4.000', 'Superficie Construida': '600', 'Última Actualización': 'Anuncio actualizado el 25/10/2024', 'Consumo Energético': 'N/A', 'Emisiones CO2': 'N/A', 'Características': [('Superficie construida:', '600 m²'), ('Superficie útil:', '600 m²'), ('Habitaciones:', '6'), ('Baños:', '7'), ('Tipo de casa:', 'Unifamiliar'), ('Antigüedad:', 'Entre 5 y 10 años'), ('Conservación:', 'En buen estado'), ('Referencia:', '2805-002317')], 'Tipo de operación': 'Alquiler'}
Inmueble 2: {'Enlace': 'https://www.pisos.com/alquilar/atico-fuencarral_el_pardo_las_tablas28050-48426361500_100500/', 'Precio': '3.500', 'Superficie Construida': '350', 'Última Actualización': 'Anuncio actualizado el 25/10/2024', 'Consumo Energético': 'N/A', 'Emisiones CO2': 'N/A', 'Características': [('Superficie construida:', '350 m²'), ('Superficie útil:', '280 m²'), ('Habitaciones:', '4'), ('Baños:', '

In [3]:
df

,Enlace,Precio,Superficie Construida,Última Actualización,Consumo Energético,Emisiones CO2,Características,Tipo de operación
0,https://www.pisos.com/alquilar/casa_unifamilia...,4.000,600,Anuncio actualizado el 25/10/2024,N/A,N/A,"[(Superficie construida:, 600 m²), (Superficie...",Alquiler
1,https://www.pisos.com/alquilar/atico-fuencarra...,3.500,350,Anuncio actualizado el 25/10/2024,N/A,N/A,"[(Superficie construida:, 350 m²), (Superficie...",Alquiler
2,https://www.pisos.com/alquilar/piso-la_finca-4...,5.500,190,Anuncio actualizado el 25/10/2024,Consumo:15 kWh/m² año,Emisiones:2 Kg CO₂/m² año,"[(Superficie construida:, 190 m²), (Superficie...",Alquiler
3,https://www.pisos.com/alquilar/piso-jeronimos2...,3.750,107,Anuncio actualizado el 22/10/2024,Consumo:138 kWh/m² año,Emisiones:34 Kg CO₂/m² año,"[(Superficie construida:, 107 m²), (Superficie...",Alquiler
4,https://www.pisos.com/alquilar/casa_adosada-se...,3.600,448,Anuncio actualizado el 21/10/2024,N/A,N/A,"[(Superficie construida:, 448 m²), (Habitacion...",Alquiler
...,...,...,...,...,...,...,...,...
295,https://www.pisos.com/alquilar/piso-carabanche...,1.075,62,Anuncio actualizado el 10/10/2024,N/A,N/A,"[(Superficie construida:, 62 m²), (Superficie ...",Alquiler
296,https://www.pisos.com/alquilar/estudio-parque_...,1.147,50,Anuncio actualizado el 03/07/2024,N/A,N/A,"[(Superficie construida:, 50 m²), (Baños:, 1),...",Alquiler
297,https://www.pisos.com/alquilar/piso-arroyos_y_...,850,50,Anuncio actualizado el 21/10/2024,N/A,N/A,"[(Superficie construida:, 50 m²), (Habitacione...",Alquiler
298,https://www.pisos.com/alquilar/piso-valdemarin...,2.200,70,Anuncio actualizado el 25/10/2024,N/A,N/A,"[(Superficie construida:, 70 m²), (Habitacione...",Alquiler
